#### Structual Equasion Modeling (SEM)

Covered
- Data cleaning of annual and monthly consumption and production data respectivly (US Energy).
- Temporal Disaggregation to High-Frequency Data of the annual consumtion data denton-cholette in R
- Index Price extraction though the library yfinance from Yahoo Finance
- Kilian Index of Global Real Economic Activity from Fed (https://fred.stlouisfed.org/series/IGREA/1000)
- https://www.eia.gov/dnav/pet/pet_pri_spt_s1_d.htm - Brent EUrope and WTI, more robust data shource requred
- All_Country_Data.xlsx - cleaning requred for the Uncertanty index (https://www.policyuncertainty.com/equity_uncert.html)


SEM
- Measurement Model
- Structual Model
- Normalization was done by deviding each column by its max values in order to standardize the scale from 0 to 1 for all outputs


In [1]:
import semopy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
## Annual Consumption

df = pd.read_csv("Data_Extract/Total_Oil_Consumption.csv", skiprows=[0],na_values=['--', 'ie'])
df.rename(columns={'Unnamed: 1': 'Country'}, inplace=True)

# df['Unnamed: 1'].name = 'Country' # 'API', 'Unnamed: 1'
df = df.drop(['API'], axis=1)
df.isnull().sum()

df = df.drop(df.index[0:2])
# df

# Transpose the df
df.set_index('Country',inplace=True)
df_transposed = df.transpose()
df_transposed.reset_index(inplace=True)
df_transposed.rename(columns={'index': 'Year'}, inplace=True)

# Handle Null/NA Values
df_transposed = df_transposed.dropna(thresh=0.7*len(df_transposed), axis=1)
df_transposed = df_transposed.rename(columns=lambda x: x.strip())
df_transposed = df_transposed.drop(df_transposed.index[0:7])
df_transposed.reset_index(drop=True, inplace=True)
# df_transposed


## Usefull in the future with some eddits
# def is_string(x):
#     return isinstance(x, str)

# is_string_series = df_transposed.apply(is_string)
# if is_string_series.any():
#     print('The column contains string data')
# else:
#     print('The column does not contain string data')


numeric_df = df_transposed.select_dtypes(include=np.number)
median = numeric_df.median(skipna=True)
df_transposed.fillna(median, inplace=True)
# df_transposed

# Show all Rows
pd.set_option('display.max_rows', 70)
summary = df_transposed.describe().T

df_transposed = df_transposed[(df_transposed['Year'] > '1996') & (df_transposed['Year'] < '2024')]
df_transposed.reset_index(drop=True, inplace=True)
df_transposed = df_transposed[['Year','China', 'India', 'Japan', 'United States']] # Germany
df_transposed
# df_transposed.isnull()
# df_transposed.iloc(43)

# Checking the the higest production on average

# Export data for the SEM Modeling
df_transposed.to_csv('Final_Working_Files/Annual_Oil_Consumption.csv') 
df_transposed 



Country,Year,China,India,Japan,United States
0,1997,3636.500000,1766.000000,5636.561645,18620.320548
1,1998,4118.000000,1758.000000,5448.019177,18917.178082
2,1999,4359.000000,1960.000000,5589.405479,19519.304110
3,2000,4688.535000,2178.336803,5458.778689,19701.005464
4,2001,4814.297000,2171.995534,5360.983563,19648.657534
5,2002,5211.508000,2256.177973,5268.676712,19761.254795
6,2003,5786.183000,2355.914822,5380.073972,20033.643836
7,2004,6802.690000,2476.343470,5267.846996,20731.122951
8,2005,6970.258000,2465.435233,5287.534246,20802.161123
9,2006,7493.351000,2654.001452,5155.961643,20687.417556


In [3]:
## Monthly Production

pd.set_option('display.max_rows', 70)
# pd.options.display.float_format = '{:,}'.format
# pd.set_option("styler.format.thousands", ",")
# pd.reset_option("styler.format.thousands")

# INT-Export-05-18-2024_20-01-44.csv
df = pd.read_csv("Data_Extract/Total_Oil_Production.csv", skiprows=[0],na_values='--')
df.rename(columns={'Unnamed: 1': 'Country'}, inplace=True)
df = df.drop(['API'], axis=1)

df['Country'] = df['Country'].apply(lambda x: x.strip())
new = df.iloc[::8, 0].copy().reset_index().drop('index',axis=1)


df = df[df['Country'] == 'Crude oil including lease condensate (Mb/d)']
df = df.reset_index().drop('index',axis=1)
df["Country"] = new #.str.cat(new, sep =" ")

# Transpose the df
df.set_index('Country',inplace=True)
df_transposed = df.transpose()
df_transposed.reset_index(inplace=True)
df_transposed.rename(columns={'index': 'Year'}, inplace=True)

# Handle Null/NA Values (probably it is better to do it after filtering for year but we get a lot of itmes with zeros and they are not significat countries)
df_transposed = df_transposed.dropna(thresh=0.7*len(df_transposed), axis=1)
df_transposed = df_transposed.rename(columns=lambda x: x.strip())

# Seperate the Year and the Month in to differnt columns and encode them
df_transposed[['Month', 'Year']] = df_transposed.Year.str.split(" ", expand = True)
month_col = df_transposed.pop('Month')
df_transposed.insert(0, 'Month', month_col)
df_transposed = df_transposed.drop('World',axis=1)

df_transposed = df_transposed[(df_transposed['Year'] > '1996') & (df_transposed['Year'] < '2024')]
df_transposed.reset_index(drop=True, inplace=True)

import calendar

d = dict((v,k) for k,v in enumerate(calendar.month_abbr))
df_transposed.Month = df_transposed.Month.map(d)
# df_transposed
# df_transposed.dtypes

# Median NA Fill
numeric_df = df_transposed.select_dtypes(include=np.number)
median = numeric_df.median(skipna=True)
df_transposed.fillna(median, inplace=True)

# Checking the the higest production on average
# summary = df_transposed.describe().T
# summary.sort_values(['mean'], ascending=False).groupby('mean').head(10)
# df_transposed

# Normalization
df_transposed.iloc[:, 2:].apply(lambda x: x / x.max())


df_transposed = df_transposed.drop(['Month', 'Year'], axis=1)
df_transposed.reset_index(drop=True, inplace=True)
max_value = df_transposed.max()
df_transposed = df_transposed.div(max_value)
df_transposed = df_transposed[['Algeria', 'Iran', 'Nigeria', 'Kuwait', 'Saudi Arabia', 'Venezuela', 'United Arab Emirates']] # 'Russia'

# df_transposed
# df_transposed.isnull().sum()

# Export data for the SEM Modeling 
df_transposed.to_csv('Final_Working_Files/Monthly_Oil_Production.csv')
df_transposed  


Country,Algeria,Iran,Nigeria,Kuwait,Saudi Arabia,Venezuela,United Arab Emirates
0,0.499906,0.818785,0.872601,0.589933,0.678311,0.901484,0.627621
1,0.488212,0.817680,0.873449,0.589933,0.678311,0.910979,0.630288
2,0.496983,0.762431,0.850366,0.586727,0.674072,0.916320,0.627755
3,0.494059,0.829834,0.882466,0.583520,0.669832,0.927893,0.628792
4,0.496983,0.769061,0.876416,0.570696,0.677039,0.936795,0.583485
...,...,...,...,...,...,...,...
319,0.672388,0.826771,0.544484,0.854206,0.754621,0.228487,0.811444
320,0.678235,0.830105,0.579166,0.867030,0.797015,0.221068,0.813866
321,0.684082,0.838294,0.586873,0.847793,0.784297,0.221068,0.816288
322,0.684082,0.874513,0.563752,0.857412,0.780058,0.225519,0.806599


In [4]:
## Economic Activity Data finalization

import pandas as pd
import datetime
import yfinance as yf

## Declaring Tickers, and dependencies for the yf.download
tickers = ['^FTSE', '^GSPC', '000001.SS']
start_date = datetime.date(1997, 1, 1)
end_date = datetime.date(2023, 12, 31)
multi_data = yf.download(tickers, start=start_date, end=end_date, period="1mo")
multi_data = multi_data['Close'].reset_index()
# multi_data


## Beacause not all Last business days are working around the world I decided to use the last available day price data as a proxy for the monthly closing price
## The below does just that 
multi_data.fillna(method='ffill', inplace=True)

## If the 1mo does not function correctly use this to extract the last working day of the month
list_bm = pd.date_range(multi_data['Date'].min(), multi_data['Date'].max(), freq='BME').tolist()
md_mo_close = multi_data.loc[multi_data['Date'].isin(list_bm)]

## Export to csv
md_mo_close.to_csv('Stock_Market_Data2.csv')  
md_mo_close



## Combining Economic Activity Data

df_kilian = pd.read_csv("Data_Extract/Kilian Index of Global Real Economic Activity.csv", na_values='--')
df_kilian = df_kilian[(df_kilian['DATE'] > '1997') & (df_kilian['DATE'] < '2024')]
df_kilian.reset_index(drop=True, inplace=True)
# df.rename(columns={'Unnamed: 1': 'Country'}, inplace=True)
md_mo_close.reset_index(drop=True, inplace=True)
md_mo_close = md_mo_close.drop(['Date'], axis=1)
md_mo_close['Kilian'] = df_kilian['IGREA']
# md_mo_close
# Normalization
# md_mo_close.max()
# md_mo_close = md_mo_close.apply(lambda x: x / x.max())
# md_mo_close['000001.SS'] = md_mo_close['000001.SS'] / md_mo_close['000001.SS'].max()
# md_mo_close.isnull().sum()
md_mo_close.to_csv('Economic_Activity.csv')  
md_mo_close
# md_mo_close.max()


[*********************100%%**********************]  3 of 3 completed
C:\Users\Darko\AppData\Local\Temp\ipykernel_23096\3011513812.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  multi_data.fillna(method='ffill', inplace=True)


Ticker,000001.SS,^FTSE,^GSPC,Kilian
0,NaN,4275.799805,786.159973,-3.791781
1,NaN,4308.299805,790.820007,-5.469825
2,NaN,4312.899902,757.119995,-3.027602
3,NaN,4436.000000,801.340027,-9.309717
4,NaN,4621.299805,848.280029,-16.743878
...,...,...,...,...
319,3119.879883,7439.100098,4507.660156,-39.943982
320,3110.475098,7608.100098,4288.049805,-20.966809
321,3018.770996,7321.700195,4193.799805,8.461345
322,3029.673096,7453.799805,4567.799805,6.466895


In [5]:
## Preparing the Economics_Policy_Uncertainty_Index


df_epui = pd.read_csv("Data_Extract/Econom_Policy_Uncertainty_Index_All_Country_Data.csv", na_values='--')

df_epui = df_epui[(df_epui['Year'] > 1996) & (df_epui['Year'] < 2024)]
df_epui[['Year', 'Month']] = df_epui[['Year', 'Month']].astype('int64') 


# Create a boolean array indicating which columns contain the string "Email"
cols_to_drop = df_epui.columns[df_epui.columns.str.contains('Unnamed')]

# Drop the columns containing the string "Unnamed" 
df_epui.drop(cols_to_drop, axis=1, inplace=True)
# cols = "China, France, Germany, United Kingdom, United States".split(sep='')
# df_epui
df_epui = df_epui[['SCMP China', 'France', 'Germany', 'UK', 'US']]

# df_epui.isna().sum()
# df_epui.fillna(method='ffill', inplace=True)

df_epui.reset_index(drop=True, inplace=True)

# df_epui = df_epui.apply(lambda x: x / x.max())

df_epui.to_csv('Final_Working_Files/Economics_Policy_Uncertainty_Index.csv')  
df_epui

,SCMP China,France,Germany,UK,US
0,88.323707,88.764135,86.682216,59.411705,75.445516
1,115.348679,68.004674,56.295875,57.932700,92.086141
2,59.410586,62.072923,47.050636,95.837305,70.452693
3,123.271347,93.868848,101.568061,95.512367,71.104439
4,20.579830,82.899866,76.708151,55.223397,75.869499
...,...,...,...,...,...
319,709.881991,152.655456,661.260914,267.425098,76.844224
320,819.746146,243.347756,737.681932,255.427358,117.698777
321,603.739894,261.824487,669.565170,270.910559,133.594075
322,743.397577,282.455855,626.137039,250.733524,140.677204


In [6]:
# War Data

war = pd.read_csv("Data_Extract/War_Annual.csv", na_values='--')
war = war[['country', 'year', 'intwar']]

war = war.pivot(index = 'year', columns = 'country', values = 'intwar')
war

war = war[['Iran', 'Kuwait', 'Saudi Arabia', 'Iraq']]
war.reset_index(inplace=True)

war = war[war['year'] > 1996]
war.reset_index(drop=True, inplace=True)

i = 2018

while i < 2023:
    war.loc[-1] = [str(i + 1), 0, 0, 0, 0] # adding a row
    war.reset_index(drop=True, inplace=True)
    # war.index = war.index + 1  # shifting index
    # war = war.sort_index()  # sorting by index
    i = i + 1


war = war.drop(['year'], axis=1)
war.to_csv('Final_Working_Files/War.csv') 
war
# war[[]]
# war[war['YEAR'] == '2018']


country,Iran,Kuwait,Saudi Arabia,Iraq
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,6.0
7,0.0,0.0,0.0,6.0
8,0.0,0.0,0.0,6.0
9,0.0,0.0,0.0,6.0


In [117]:
# WTI and Brent

wti = pd.read_csv('Data_Extract/WTI.csv', skiprows=4)
brent = pd.read_csv('Data_Extract/Brent.csv',skiprows=4)
brent
wti

### WTI

wti.rename(columns={'Month': 'Year'}, inplace=True)
wti.rename(columns={'Cushing OK WTI Spot Price FOB Dollars per Barrel': 'WTI'}, inplace=True)
wti[['Month', 'Year']] = wti.Year.str.split(" ", expand = True)
month_col = wti.pop('Month')
wti.insert(1, 'Month', month_col)
# wti = wti.drop('World',axis=1)


wti.reset_index(drop=True, inplace=True)

import calendar

d = dict((v,k) for k,v in enumerate(calendar.month_abbr))
wti.Month = wti.Month.map(d)
wti = wti.sort_values(["Year", "Month"])
wti = wti[(wti['Year'] > '1996') & (wti['Year'] < '2024')]
wti.reset_index(drop=True, inplace=True)

### Brent

brent.rename(columns={'Month': 'Year'}, inplace=True)
brent.rename(columns={'Europe Brent Spot Price FOB Dollars per Barrel': 'Brent'}, inplace=True)
brent[['Month', 'Year']] = brent.Year.str.split(" ", expand = True)
month_col = brent.pop('Month')
brent.insert(1, 'Month', month_col)
# wti = wti.drop('World',axis=1)


brent.reset_index(drop=True, inplace=True)

import calendar

d = dict((v,k) for k,v in enumerate(calendar.month_abbr))
brent.Month = brent.Month.map(d)
brent = brent.sort_values(["Year", "Month"])
brent = brent[(brent['Year'] > '1996') & (brent['Year'] < '2024')]
brent.reset_index(drop=True, inplace=True)




PoOil = pd.concat([wti['WTI'], brent['Brent']], axis=1)
PoOil.to_csv('Final_Working_Files/PoOil.csv')
PoOil
# df_concat.columns = df_concat.columns.str.replace(' ', '.')

,WTI,Brent
0,25.13,23.54
1,22.18,20.85
2,20.97,19.13
3,19.70,17.56
4,20.82,19.02
...,...,...
319,81.39,86.15
320,89.43,93.72
321,85.64,90.60
322,77.69,82.94


In [7]:
### Preparing the data for SEM direct input

econ_act = pd.read_csv("Final_Working_Files/Economic_Activity.csv")
econ_uncer_idx = pd.read_csv("Final_Working_Files/Economics_Policy_Uncertainty_Index.csv")
consumption = pd.read_csv("Final_Working_Files/Monthly_Oil_Consumption.csv")
production = pd.read_csv('Final_Working_Files/Monthly_Oil_Production.csv')

PoOil = pd.read_csv('Final_Working_Files/PoOil.csv')

df_dict = {
    'econ_act': econ_act, 
    'econ_uncer_idx': econ_uncer_idx, 
    'consumption': consumption, 
    'production': production,
    'PoOil': PoOil
    }


for key, df in df_dict.items():
    df.columns = [str(key) + "." + str(i) for i in df.columns] 
df_concat = pd.concat([econ_act, econ_uncer_idx, consumption, production, PoOil], axis=1)

df_concat.columns = df_concat.columns.str.replace(' ', '.')

# Create a boolean array indicating which columns contain the string "Unnamed"
cols_to_drop = df_concat.columns[df_concat.columns.str.contains('Unnamed')]

# Drop the columns containing the string "Unnamed" 
df_concat.drop(cols_to_drop, axis=1, inplace=True)

df_concat = df_concat.apply(lambda x: x / x.max())

df_concat['PoOil.OilComb'] =  df_concat['PoOil.WTI'] - df_concat['PoOil.Brent'] 
df_concat = df_concat.drop(['PoOil.WTI', 'PoOil.Brent'], axis=1)

df_concat.to_csv('Final_Working_Files/SME_input.csv')
df_concat


,econ_act.000001.SS,econ_act.^FTSE,econ_act.^GSPC,econ_act.Kilian,econ_uncer_idx.SCMP.China,econ_uncer_idx.France,econ_uncer_idx.Germany,econ_uncer_idx.UK,econ_uncer_idx.US,consumption.China,...,consumption.Japan,consumption.United.States,production.Algeria,production.Iran,production.Nigeria,production.Kuwait,production.Saudi.Arabia,production.Venezuela,production.United.Arab.Emirates,PoOil.OilComb
0,NaN,0.542869,0.164819,-0.020035,0.090978,0.154471,0.102600,0.052034,0.149704,0.207050,...,1.000000,0.881769,0.499906,0.818785,0.872601,0.589933,0.678311,0.901484,0.627621,0.010340
1,NaN,0.546995,0.165796,-0.028901,0.118815,0.118345,0.066634,0.050738,0.182724,0.207338,...,0.999544,0.881849,0.488212,0.817680,0.873449,0.589933,0.678311,0.910979,0.630288,0.008573
2,NaN,0.547579,0.158731,-0.015997,0.061196,0.108022,0.055691,0.083936,0.139797,0.207913,...,0.998632,0.882009,0.496983,0.762431,0.850366,0.586727,0.674072,0.916320,0.627755,0.012495
3,NaN,0.563209,0.168002,-0.049190,0.126975,0.163354,0.120220,0.083651,0.141090,0.208775,...,0.997264,0.882249,0.494059,0.829834,0.882466,0.583520,0.669832,0.927893,0.628792,0.014838
4,NaN,0.586735,0.177843,-0.088470,0.021198,0.144266,0.090794,0.048365,0.150546,0.209925,...,0.995439,0.882568,0.496983,0.769061,0.876416,0.570696,0.677039,0.936795,0.583485,0.012203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,0.523930,0.944492,0.945036,-0.211053,0.731212,0.265657,0.782692,0.234215,0.152480,0.191103,...,0.585448,0.963483,0.672388,0.826771,0.544484,0.854206,0.754621,0.228487,0.811444,-0.041178
320,0.522351,0.965949,0.898994,-0.110783,0.844377,0.423484,0.873146,0.223707,0.233546,0.164478,...,0.585310,0.964255,0.678235,0.830105,0.579166,0.867030,0.797015,0.221068,0.813866,-0.038162
321,0.506950,0.929586,0.879235,0.044707,0.621880,0.455638,0.792521,0.237267,0.265087,0.144509,...,0.585206,0.964835,0.684082,0.838294,0.586873,0.847793,0.784297,0.221068,0.816288,-0.042963
322,0.508781,0.946358,0.957644,0.034169,0.765734,0.491541,0.741118,0.219596,0.279142,0.131196,...,0.585137,0.965221,0.684082,0.874513,0.563752,0.857412,0.780058,0.225519,0.806599,-0.044629


### SEM Model in Python with Semopy

In [9]:


# Import model description and data:
# from semopy import Model
# from semopy.examples import example_model
# ea =~ econ_act.000001.SS + econ_act.^FTSE + econ_act.^GSPC + econ_act.Kilian
# oc ~ ea


desc = """
    # measurement model
        epu =~ econ_uncer_idx.SCMP.China + econ_uncer_idx.France + econ_uncer_idx.Germany + econ_uncer_idx.UK + econ_uncer_idx.US
        
        oc =~ consumption.China + consumption.India + consumption.Japan + consumption.United.States
        op =~ production.Algeria + production.Iran + production.Nigeria + production.Kuwait + production.Saudi.Arabia + production.Venezuela + production.United.Arab.Emirates
        PoOil =~ PoOil.WTI + PoOil.Brent
    # regressions
        
        PoOil ~ epu + op + oc 
    """
data = pd.read_csv('Final_Working_Files/SME_input.csv')

col = [x.strip() for x in data.columns]
data.columns = col
data.columns
# col = data.columns.strip()
# col


Index(['Unnamed: 0', 'econ_act.000001.SS', 'econ_act.^FTSE', 'econ_act.^GSPC',
       'econ_act.Kilian', 'econ_uncer_idx.SCMP.China', 'econ_uncer_idx.France',
       'econ_uncer_idx.Germany', 'econ_uncer_idx.UK', 'econ_uncer_idx.US',
       'consumption.China', 'consumption.India', 'consumption.Japan',
       'consumption.United.States', 'production.Algeria', 'production.Iran',
       'production.Nigeria', 'production.Kuwait', 'production.Saudi.Arabia',
       'production.Venezuela', 'production.United.Arab.Emirates',
       'PoOil.OilComb'],
      dtype='object')

In [15]:
df_monhly_consumption = pd.read_csv("Final_Working_Files/Monthly_Oil_Consumption.csv", na_values='--')

# df_monhly_consumption = df_monhly_consumption.apply(lambda x: x / x.max())

# df_edf_monhly_consumptionpui.to_csv('FinalFinal_Working_Files/Monthly_Oil_Consumption.csv') 
# len(df_monhly_consumption['China'])

df_monhly_consumption

,China,India,Japan,United.States
0,293.334145,148.175522,474.875036,1548.349148
1,293.741454,148.133192,474.658466,1548.489465
2,294.556071,148.048533,474.225328,1548.770100
3,295.777997,147.921544,473.575621,1549.191052
4,297.407231,147.752226,472.709344,1549.752322
...,...,...,...,...
319,270.740927,143.948901,278.014585,1691.834967
320,233.020304,133.082636,277.948889,1693.191442
321,204.729836,124.932937,277.899618,1694.208799
322,185.869524,119.499804,277.866770,1694.887036
